## Work
1. 請嘗試寫一個 callback 用來記錄各類別在訓練過程中，對驗證集的 True Positive 與 True Negative

In [6]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [7]:
## 資料前處理
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
def preproc_x(x):
    x = x.astype("float32")
    x = x.reshape(x.shape[0],-1)
    x = scaler.fit_transform(x)
    return x

def preproc_y(y):
    y = keras.utils.to_categorical(y, num_classes=10)
    return y      

In [8]:
"""
建立你的神經網路
"""
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

def build_mlp(input_dim, layer_output_units, output_units, regularizer=None, dropout_ratio=None, isBN=False):
    model = Sequential()
    for i, untis in enumerate(layer_output_units):  
        if i == 0:
            model.add(Dense(units=untis, 
                            input_dim=input_dim,
                            kernel_initializer='normal',
                            kernel_regularizer=regularizer, 
                            activation='relu'))
            if isBN:
                model.add(BatchNormalization())
        else:
            model.add(Dense(units=untis,
                            kernel_initializer='normal',
                            kernel_regularizer=regularizer, 
                            activation='relu'))
            if isBN:
                model.add(BatchNormalization())

    if dropout_ratio:
        model.add(Dropout(dropout_ratio))
        
    model.add(Dense(units=output_units,
                kernel_initializer='normal', 
                activation='softmax'))
    return model  

In [9]:
"""
訓練模型
"""
def complie_train_model(model, optimizer, epochs, batch_size, callbacks,   x_train, y_train, x_test, y_test):
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer= optimizer)
    model.fit(x_train, y_train, 
          epochs= epochs, 
          batch_size=batch_size, 
          validation_data=(x_test, y_test), 
          callbacks=callbacks, 
          shuffle=True)
    return model

In [10]:
"""
以視覺畫方式檢視訓練過程
"""
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def show_train_history(results, title=""):
    for i, result in enumerate(results):
        history = result["model"].history.history
        tag = "({})".format(result["tag"])
        df_acc = pd.DataFrame({"acc"+tag:history["acc"], "val_acc"+tag:history["val_acc"]})
        df_loss = pd.DataFrame({"loss"+tag:history["loss"], "val_loss"+tag:history["val_loss"]})
        if i ==0:
            df_acc_results = df_acc.copy()
            df_loss_results = df_loss.copy()
        else:
            df_acc_results= pd.concat([df_acc_results, df_acc], axis=1)
            df_loss_results= pd.concat([df_loss_results, df_loss], axis=1)  

    fig, ax = plt.subplots(1,2, figsize=(18,5))
    styles = ['bs-','ro-','y^-', 'g--']    
    g1 = df_acc_results.plot.line(style=styles, ax =ax[0])
    g1.set_xlabel("Epoch")
    g1.set_ylabel("Acc") 
    g1.set_title(title) 
    
    g2 = df_loss_results.plot.line(style=styles, ax =ax[1])
    g2.set_xlabel("Epoch")
    g2.set_ylabel("Loss") 
    g2.set_title(title) 
    return df_acc_results, df_loss_results, g1, g2

In [12]:
"""
設定 callbacks
"""

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def setup_callbacks(ismodelckpt=False, isearlystop=False, isreducelr = False, 
                    monitor="val_acc", save_only="best", patience=5, min_lr= 1e-12, factor=0.5 ):
    cbs=list()
    
    if ismodelckpt:
        modelckpt = ModelCheckpoint(filepath="./tmp.h5",
                                    monitor=monitor, 
                                    save_best_only="best" in save_only,
                                    save_weights_only="weights" in save_only)
        cbs.append(modelckpt)
    if isearlystop:
        earlystop = EarlyStopping(monitor=monitor, 
                                  patience=patience, 
                                  verbose=1)
        cbs.append(earlystop)
    if isreducelr:
        reducelr = ReduceLROnPlateau(factor=factor, 
                              min_lr=min_lr, 
                              monitor=monitor, 
                              patience=patience, 
                              verbose=1)
        cbs.append(reducelr)
    return cbs

In [52]:
# 載入 Callbacks
from keras.callbacks import Callback
import numpy as np
from sklearn.metrics import confusion_matrix

def ravel_cm(cm):
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    return FP, FN, TP, TN

# Record_fp_tp
class Record_tp_tn(Callback):
    """
    撰寫一個紀錄 ture positive 與 true negative 數量的 callbacks    
    """
    def on_train_begin(self, epoch, logs = {}):            
        if "val_tp" not in self.params['metrics']:
            self.params['metrics'].append("val_tp")
        if "val_tn" not in self.params['metrics']:
            self.params['metrics'].append("val_tn")            
            
    def on_epoch_end(self, epoch, logs = {}, thres=0.5):
        y_true = self.validation_data[1].argmax(axis = 1)
        y_pred = self.model.predict(self.validation_data[0])   
        y_pred = (y_pred[:, 1] >= thres) * 1
        
        cm = confusion_matrix(y_true, y_pred)
        FP, FN, TP, TN = ravel_cm(cm)
        print(TP)
        print(TN)
        logs["val_tp"] = TP
        logs["val_tn"] = TN
    
rec_tptn = Record_tp_tn()

In [24]:
train, test = keras.datasets.cifar10.load_data()

In [25]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [84]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 1024
MOMENTUM = 0.95
DROPOUT_EXP = 0.25
L2_EXP = 1e-3

In [85]:
"""
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
results = list()

keras.backend.clear_session() # 把舊的 Graph 清掉
regularizer = keras.regularizers.l1_l2(l1=L2_EXP, l2=L2_EXP)
model = build_mlp(input_dim=x_train.shape[1], layer_output_units=[256,128], output_units=10, regularizer=regularizer, dropout_ratio=DROPOUT_EXP, isBN=True)
optimizer =  keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
callbacks = [Record_tp_tn()]
model = complie_train_model(model=model, optimizer=optimizer,  epochs=EPOCHS , batch_size=BATCH_SIZE, callbacks=callbacks, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)
results.append({"tag": "tptn", "model":model})

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 20s 407us/step - loss: 36.7838 - acc: 0.1686 - val_loss: 36.1393 - val_acc: 0.2754
[1000    5    0    0    0    0    0    0    0    0]
[   7 8998 9000 9000 9000 9000 9000 9000 9000 9000]
Epoch 2/25
50000/50000 [==============================] - 14s 288us/step - loss: 35.6621 - acc: 0.2929 - val_loss: 35.0908 - val_acc: 0.3350
[1000   39    0    0    0    0    0    0    0    0]
[  58 8981 9000 9000 9000 9000 9000 9000 9000 9000]
Epoch 3/25
50000/50000 [==============================] - 13s 257us/step - loss: 34.6431 - acc: 0.3350 - val_loss: 34.1170 - val_acc: 0.3579
[1000   94    0    0    0    0    0    0    0    0]
[ 132 8962 9000 9000 9000 9000 9000 9000 9000 9000]
Epoch 4/25
50000/50000 [==============================] - 14s 284us/step - loss: 33.6833 - acc: 0.3569 - val_loss: 33.1830 - val_acc: 0.3726loss: 33.99 - ETA: 5s - loss: 33.9088 - acc: 0.35 - ETA: 4s - loss: - ETA: 

In [95]:
def reduce(arr):
    a = arr[0]
    for b in arr[1:]:
        a = np.append(a , b)
    return a
    
h = results[0]['model'].history.history
val_tp = reduce(h['val_tp'])
print("val_tp")
print(val_tp)
val_tn = reduce(h['val_tn'])
print("val_tn")
print(val_tn)

val_tp
[1000    5    0    0    0    0    0    0    0    0 1000   39    0    0
    0    0    0    0    0    0 1000   94    0    0    0    0    0    0
    0    0 1000  117    0    0    0    0    0    0    0    0 1000  141
    0    0    0    0    0    0    0    0  999  157    0    0    0    0
    0    0    0    0 1000  162    0    0    0    0    0    0    0    0
 1000  173    0    0    0    0    0    0    0    0  998  201    0    0
    0    0    0    0    0    0  998  213    0    0    0    0    0    0
    0    0  998  236    0    0    0    0    0    0    0    0  996  237
    0    0    0    0    0    0    0    0  997  253    0    0    0    0
    0    0    0    0  997  264    0    0    0    0    0    0    0    0
  995  273    0    0    0    0    0    0    0    0  994  292    0    0
    0    0    0    0    0    0  994  307    0    0    0    0    0    0
    0    0  994  315    0    0    0    0    0    0    0    0  994  329
    0    0    0    0    0    0    0    0  994  342    0    0    0    0